In [1]:
import pandas as pd
import numpy as np
import re 
from fuzzywuzzy import fuzz

pd.set_option("display.max_columns", None)

# User Data Frame

#### Goals

- Use function created on 06.2_user_df notebook to get Hulu's and Disney+'s data frames from the answers filled out from the survey.

Survey Link: https://forms.gle/Jb7gzK7VMbnvpRvP9

### 1. Import data

In [2]:
genres_recommender = pd.read_pickle("Data_Hulu_Disney/genres_recommender_v2.pkl")

In [3]:
genres_recommender

genre,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
streaming_service,,,,,,,,,,,,,,,,,,,,,,,,,,
Amazon,0.1212,0.1228,0.1619,0.0191,0.2432,0.1252,0.2018,0.3349,0.1045,0.0574,0.0152,0.0606,0.0263,0.0096,0.0032,0.0582,0.0048,0.1021,0.0550,0.0327,0.0144,0.0152,0.0072,0.0287,0.0112,0.0199
HBO,0.0769,0.0533,0.0473,0.0355,0.3905,0.1538,0.1006,0.5089,0.0414,0.0473,0.0059,0.0769,0.0118,0.0237,0.0059,0.0828,0.0296,0.0237,0.0769,0.0237,0.0000,0.0533,0.0414,0.0355,0.0059,0.0000
Netflix,0.1399,0.1190,0.1675,0.0221,0.2736,0.1466,0.1718,0.3767,0.0620,0.0571,0.0123,0.0387,0.0344,0.0178,0.0061,0.0571,0.0018,0.0883,0.0785,0.0344,0.0025,0.0160,0.0110,0.0479,0.0098,0.0031
Disney,0.3061,0.4286,0.4898,0.0000,0.4422,0.0000,0.1361,0.0680,0.3469,0.0340,0.0000,0.0000,0.0000,0.0408,0.0136,0.0068,0.0000,0.1156,0.0068,0.0408,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
Hulu,0.1607,0.1251,0.2187,0.0154,0.3536,0.1251,0.0804,0.3319,0.0727,0.0566,0.0524,0.0196,0.0294,0.0377,0.0063,0.0671,0.0098,0.1670,0.0664,0.0342,0.0049,0.0147,0.0231,0.0349,0.0049,0.0049


We will need to add Hulu and Disney+'s shows to the all_shows data frame in order to have the complete list of shows.

In [4]:
all_shows = pd.read_pickle("../Data/all_shows_genres.pkl")

In [5]:
all_shows.head()

,show,genres,Crime,Drama,Thriller,Fantasy,Horror,Mystery,Comedy,Sci-Fi,Biography,Action,Adventure,Romance,History,Documentary,Animation,War,Sport,Family,Western,Short,Reality-TV,Musical,Music,Game-Show,Talk-Show,News
0,Breaking Bad,"Crime,Drama,Thriller",1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
1,Stranger Things,"Drama,Fantasy,Horror",0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
2,Sherlock,"Crime,Drama,Mystery",1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
3,Better Call Saul,"Crime,Drama",1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0
4,The Office,Comedy,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0


In [6]:
hulu = pd.read_pickle("Data_Hulu_Disney/hulu_genres.pkl")

disney = pd.read_pickle("Data_Hulu_Disney/disney_genres.pkl")

In [7]:
all_shows = all_shows.append(hulu).append(disney).reset_index(drop=True)

In [8]:
# all_shows.to_pickle("Data_Hulu_Disney/all_shows_genres_v2.pkl")

In [15]:
url = "https://docs.google.com/spreadsheets/d/1VsL_x7WAcEm-pLNmCqNwrDlFjTR_PUDpiadyUYJM-AQ/export?format=csv&gid=1360685521"
survey_df = pd.read_csv(url)

### 2. Creat user data frame and export

We will use the function created on 06.2_user_df notebook.

In [10]:
def get_recommender_df(survey_df, df_all_shows):
    
    genre_columns = ['Crime', 'Drama', 'Thriller', 'Fantasy', 'Horror', 'Mystery', 'Comedy', 'Sci-Fi', 'Biography',
                 'Action', 'Adventure', 'Romance', 'History', 'Documentary', 'Animation', 'War', 'Sport',
                 'Family', 'Western', 'Short', 'Reality-TV', 'Musical', 'Music', 'Game-Show', 'Talk-Show', 'News']
    
    survey_df = survey_df.drop(columns="Timestamp")
    
    all_shows_lst = [show for show in df_all_shows["show"]]
    
    titles = []
        
    # we will select just the last element using the tail pandas method
    for show in survey_df.tail(1).values[0]:
        # remove the year and description
        title_pattern = r"(.*)\s\(\d{4}\).*$"
        title = re.findall(title_pattern, show)
        titles.append(title[0])

    matches = []

    for title in titles:
        for show in all_shows_lst:
            ratio = fuzz.ratio(title.lower(), show.lower())
            if ratio == 100:
                matches.append(show)
                matches = list(set(matches))
                              
    user_df = df_all_shows[df_all_shows["show"].isin(matches)].reset_index(drop=True)
    
    user_genre_ratio = (pd.DataFrame(user_df[genre_columns].sum()
                                    .sort_values(ascending=False))
                       .reset_index()
                       .rename(columns={"index":"genre", 0:"frequency"}))

    user_genre_ratio["ratio"] = round((user_genre_ratio["frequency"] / 30), 4)

    user_genre_ratio["user"] = "User"

    user_recommender = user_genre_ratio.pivot_table(index="user",
                                          columns="genre",
                                          values="ratio")
    
    return user_recommender

We will get the user data frames for Hulu and Disney test, as done previously for Netflix, Amazon and HBO. We will test these data frames on the next notebook (05.3_streaming_service_recommender_model).

In [16]:
user_hulu = get_recommender_df(survey_df, all_shows)

In [12]:
# user_hulu.to_pickle("Data_Hulu_Disney/hulu_test.pkl")

In [13]:
url = "https://docs.google.com/spreadsheets/d/1VsL_x7WAcEm-pLNmCqNwrDlFjTR_PUDpiadyUYJM-AQ/export?format=csv&gid=1360685521"
survey_df = pd.read_csv(url)

In [17]:
user_disney = get_recommender_df(survey_df, all_shows)

In [20]:
# user_disney.to_pickle("Data_Hulu_Disney/disney_test.pkl")